# Извлечение ключевых слов

## Часть первая. Датасет

Нам нужны тексты с ключевыми словами. Мы можем взять датасет со статьями с хабра. Статьи длмнные, но обычно посвящены одной широкой теме или нескольким узким. Возможно, эти данные будет проще анализировать, так как ресурс знакомый и читаемый.

In [27]:
import re

In [28]:
#!pip install jsonlines

^C


In [29]:
import jsonlines

linelist = []
with jsonlines.open('habrahabr.jsonlines') as f:
    for obj in f:
        linelist.append(obj)

In [77]:
#len(linelist)

In [78]:
#print(linelist[0])

## Часть вторая. Ручная разметка и эталон

In [30]:
tokens = 0
texts = []
keywords_origin = []
for i in range(5):
    texts.append(linelist[i]['content'])
    keywords_origin.append(linelist[i]['keywords'])
    tokens += len(linelist[i]['content'].split())
print(tokens)

13085


In [31]:
keywords_of_markup = []
keywords_of_markup.append(['MassTransit', 'open source', '.NET', 'команды', 'события'])
keywords_of_markup.append(['XenForo', 'плагины', 'форумные движки', 'геймификация', 'форум'])
keywords_of_markup.append(['Postgresql', 'failover', 'standby', 'master', 'repmgr', 'кластер'])
keywords_of_markup.append(['SQL', 'синтаксический анализатор', 'транслятор', '1C', 'Irony'])
keywords_of_markup.append(['аудиотехнологии', 'амплитудно-частотные характеристики', 'музыка', 'аудиосистемы', 'наушники'])

In [32]:
import pandas as pd
dataset_of_texts = pd.DataFrame()
dataset_of_texts['text'] = texts
dataset_of_texts['original_keywords'] = keywords_origin
dataset_of_texts['markup_keywords'] = keywords_of_markup
dataset_of_texts

,text,original_keywords,markup_keywords
0,"MassTransit это open source библиотека, разра...","[.net, rabbitmq, masstransit]","[MassTransit, open source, .NET, команды, собы..."
1,Введение и выбор решения \r\nРано или поздно ...,"[геймификация, xenforo, форумные движки, форум...","[XenForo, плагины, форумные движки, геймификац..."
2,\r\nНа сегодняшний день процедура реализации ...,"[postgresq, haproxy, pgbouncer, keepalived, re...","[Postgresql, failover, standby, master, repmgr..."
3,"Как часто, программируя очередную бизнес-фичу,...","[irony, .net, c#, грамматический разбор, синта...","[SQL, синтаксический анализатор, транслятор, 1..."
4,"Индустрия звука, о которая была у всех на слух...","[аудиомания, мифы и реальность, акустика, ауди...","[аудиотехнологии, амплитудно-частотные характе..."


С одной стороны, размеченные и изначальные ключевые слова похожи и местами совпадают, но не везде. Например, одно и то же слово может быть написано капсом или нет, или это могут быть одинаковые термины из русского и английского. Размеченные слова следуют в основном за технической стороной статьи, тогда как оригинальные могут быть довольно широкими: например, "мифы и реальность" или "бухгалтеры и программисты". Кроме того, в исходной разметке больше слов, даже если они очень похожи и на ту же тему. Кажется, что наилучшим исходом будет объединение ключевых слов. Итак, объединим.

In [33]:
keywords_standard = []
for i in range(5):
    keywords_standard.append(set(keywords_origin[i]) | set(keywords_of_markup[i]))
dataset_of_texts['standard_keywords'] = keywords_standard

In [34]:
dataset_of_texts

,text,original_keywords,markup_keywords,standard_keywords
0,"MassTransit это open source библиотека, разра...","[.net, rabbitmq, masstransit]","[MassTransit, open source, .NET, команды, собы...","{.NET, события, .net, MassTransit, команды, ma..."
1,Введение и выбор решения \r\nРано или поздно ...,"[геймификация, xenforo, форумные движки, форум...","[XenForo, плагины, форумные движки, геймификац...","{xenforo, форумные движки, gamification, гейми..."
2,\r\nНа сегодняшний день процедура реализации ...,"[postgresq, haproxy, pgbouncer, keepalived, re...","[Postgresql, failover, standby, master, repmgr...","{repmgr, keepalived, postgresq, Postgresql, pg..."
3,"Как часто, программируя очередную бизнес-фичу,...","[irony, .net, c#, грамматический разбор, синта...","[SQL, синтаксический анализатор, транслятор, 1...","{Irony, анализатор кода, SQL, 1C, irony, регул..."
4,"Индустрия звука, о которая была у всех на слух...","[аудиомания, мифы и реальность, акустика, ауди...","[аудиотехнологии, амплитудно-частотные характе...","{аудиотехника, наушники, аудиомания, акустика,..."


Чудесно, всё объединилось.  А теперь можно попробовать методы автоматического ввыделения ключевых слов

## Часть третья. Автоматическое выделение ключевых слов

### Препроцессинг

И здесь-то нас и подстерегают все сложности, которые есть у этого датасета. Он одновременно на русском, с использованием некоторых английских терминов, которые при этом являются основополагающими, да ещё и с добавлением кода, который, конечно, вряд ли должен быть в ключевых словах -- но вот незадача, он местами очень похож на английский! Так что ждут нас проблемы и наши тексты весьма сложно формализуемы.

In [35]:
import stanza
#stanza.download('ru')
#stanza.download('en')

nlp_ru = stanza.Pipeline(lang='ru', processors='tokenize,lemma,pos')
nlp_en = stanza.Pipeline(lang='en', processors='tokenize,lemma,pos')

2021-11-07 15:32:20 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |
| lemma     | syntagrus |

2021-11-07 15:32:20 INFO: Use device: cpu
2021-11-07 15:32:20 INFO: Loading: tokenize
2021-11-07 15:32:20 INFO: Loading: pos
2021-11-07 15:32:22 INFO: Loading: lemma
2021-11-07 15:32:22 INFO: Done loading processors!
2021-11-07 15:32:22 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |

2021-11-07 15:32:22 INFO: Use device: cpu
2021-11-07 15:32:22 INFO: Loading: tokenize
2021-11-07 15:32:22 INFO: Loading: pos
2021-11-07 15:32:24 INFO: Loading: lemma
2021-11-07 15:32:24 INFO: Done loading processors!


In [80]:
def prepocess(text, nlp):
    doc = nlp(text)
    stops = ['ADP', 'AUX', 'CCONJ', 'DET', 'INTJ', 'PART', 'PUNCT', 'SCONJ']
    final_text = []
    for sent in doc.sentences:
        sent_list = []
        count_foreing = 0
        for word in sent.words:
            #print(word)
            if word.upos not in stops:
                if not re.search('[\\\/&{}_]+?', word.text):
                    sent_list.append(word.lemma.lower())
                if word.feats:
                    if 'Foreign=Yes' in word.feats:
                        count_foreing += 1
                    
        if count_foreing < 10:
            final_text.extend(sent_list)
    return ' '.join(final_text)

Что тут, собственно, происходит.  Мы берём текст, и для каждого предложения смотрим, а не код ли оно. Если в предложении меньше десяти английских слов, то ладно, не код.
Мы можем так задеть какие-то части предложений или, наоборот, оставить маленькие кусочки кода, но в целом ситуация должна стать лучше

In [63]:
prepocess('get', nlp_ru)

'get'

In [64]:
clean_texts = []
for text in dataset_of_texts['text']:
    clean_texts.append(prepocess(text, nlp_ru))
clean_keywords = []
for keywords in dataset_of_texts['standard_keywords']:
    words = []
    for word in keywords:
        words.append(prepocess(word, nlp_ru))
    clean_keywords.append(words)

In [65]:
clean_texts[3]

'часто программировать очередной бизнес фича вы ловить себя мысль быть земля человек который писать база данные распознавать лицо фотография делать фреймворка реализовать интересный алгоритм почему работа все сводиться перекладывание один таблица бд другой вызов http сервис верстка html форма прочий «бизнес лапший мочь быть я заниматься что-то то работать компания хороший новость то интересный задача окружать мы повсюду сильный желание смелость творить чудо путь цель задача масштаб стать вы сила стоить начать она делать недавно мы писать синтаксический анализатор язык запрос 1с транслятор обычный sql это позволить мы выполнять запрос 1с участие 1с минимальный рабочий версия regexp-az получиться неделя два еще месяц уйти полноценный парсер грамматика разгребание нюанс структура бд разный 1с объект реализация специфический оператор функция результат решение поддерживать практически конструкция язык исходный код выложить github кат мы рассказать зачем мы это понадобиться удаться так затро

In [66]:
clean_texts_keywords = pd.DataFrame()
clean_texts_keywords['texts'] = clean_texts
clean_texts_keywords['standard_keywords'] = clean_keywords

### RAKE

In [52]:
#!pip install python-rake

In [67]:
import RAKE
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords

In [68]:
stop = stopwords.words('russian')

In [69]:
rake = RAKE.Rake(stop)

In [70]:
rake_res = []
for text in clean_texts_keywords['texts']:
    text_res = []
    kw_list = rake.run(text, maxWords=3, minFrequency=1)
    for el in kw_list:
        if el[1] != 0:
            text_res.append(el)
            print(el)
    rake_res.append(text_res)
    print('---------------------------------')
clean_texts_keywords['rake'] = rake_res

('receiveendpoint host e', 9.0)
('type type пример', 9.0)
('формировать отправлять событие', 9.0)
('получить сообщение', 4.0)
('выбор консьюмер', 4.0)
('статья рассказать', 4.0)
('newline console', 4.0)
('находиться имплементация', 4.0)
('схожий имплементация', 4.0)
('write', 1.0)
('environment', 1.0)
('smssent', 1.0)
('datetime', 1.0)
('start', 1.0)
---------------------------------
('получить внутри badge', 9.0)
('создавать трофей трофей', 7.833333333333332)
('определить трофей', 4.666666666666666)
('создавать включая', 4.5)
('бывать коммерческий', 4.0)
('поле низко', 4.0)
('интерес представлять', 4.0)
('важный первый', 4.0)
('пробовать оценить', 4.0)
('далее', 1.0)
('таба', 1.0)
('комбинировать', 1.0)
('видный', 1.0)
('детально', 1.0)
---------------------------------
('мастер-нод восстановление', 9.0)
('ssh спрашивать доверять', 9.0)
('ока идти далеко', 9.0)
('создать файл current_master', 9.0)
('предыдущий мастер который', 9.0)
('выбрать редакактировать pg_hba', 9.0)
('pgbouncer м

### TextRank

In [53]:
#!pip install summa

In [71]:
from summa import keywords

In [72]:
textrank_res = []
for text in clean_texts_keywords['texts']:
    text_res = []
    kw_list = keywords.keywords(text, language='russian', additional_stopwords=stop, scores=True)
    for el in kw_list:
        if el[1] != 0:
            text_res.append(el)
            print(el)
    textrank_res.append(text_res)
    print('---------------------------------')
clean_texts_keywords['textrank'] = textrank_res

('сообщение', 0.37089808371824723)
('консьюмер', 0.21540568248167388)
('тип', 0.19213232105415434)
('команда событие', 0.1886906683440217)
('exchange', 0.17738269892650924)
('наименование', 0.12990296082773795)
('процесс который', 0.12529661247235505)
('код', 0.11438941823881178)
('случай', 0.10718448571750971)
('библиотека использовать', 0.10432253055681093)
('конфигурация контейнер', 0.10258829363998175)
('masstransit', 0.10173403560110136)
('следующий', 0.0940768143863057)
('документация', 0.08985492376637803)
('ibus', 0.08975427852503873)
('обработка', 0.08937237401566817)
('метод publish интерфейс', 0.08926386921170758)
('данные', 0.08820661304809732)
('данный', 0.08820661304809732)
('имя', 0.08001408757707883)
('bus', 0.0800095142583262)
('качество message', 0.07946093823920886)
('await', 0.0788254629388799)
('подключение', 0.07810810062540105)
('ibuscontrol', 0.077988055602667)
('send', 0.07791638467438915)
('осуществлять', 0.0764668957528011)
('осуществляться', 0.07646689575280

('звук который', 0.20409436332084108)
('система', 0.18984144729663463)
('мочь', 0.17200882567262354)
('использовать', 0.16680332557285493)
('аудиотехника', 0.1581925822017473)
('усилитель', 0.15542982194705401)
('звучание', 0.14176394145859983)
('профессиональный', 0.12992110447248376)
('это', 0.12449155155149734)
('акустика наушник характеристика дело', 0.11449979361173618)
('ответ', 0.10662125083480761)
('ответить', 0.10662125083480761)
('устройство', 0.10042632759575959)
('дорогой', 0.09664695351084929)
('иной аудиосистема', 0.0963182004081625)
('обычный', 0.09453148165682722)
('обычно', 0.09453148165682722)
('качественный', 0.08818324519194254)
('стоить', 0.08714814311194116)
('качество', 0.08633478412077109)
('задача', 0.08234102349682194)
('специальный', 0.08128209688523755)
('вредный', 0.0809526140757929)
('вредний', 0.0809526140757929)
('цап', 0.08090998931571801)
('вред', 0.07819090739820442)
('обладать', 0.07737454145363437)
('внутриканальный', 0.0769777479265289)
('современн

### Tf-Idf

## Часть четвёртая. Морфологические и синтаксические шаблоны

Давайте посмотрим на наши эталонные списки

In [73]:
for i in range(5):
    print(clean_texts_keywords['standard_keywords'][i])

['net', 'событие', '.net', 'masstransit', 'команда', 'masstransit', 'open source', 'rabbitms']
['xenforo', 'форумный движок', 'gamification', 'геймификация', 'xenforo', 'привлечение пользователь', 'форум', 'плагин']
['repmgr', 'keepalived', 'postgresq', 'postgresql', 'pgbouncer', 'replication', 'ha', 'haproxy', 'failover', 'cluster', 'кластер', 'standby', 'master']
['irony', 'анализатор код', 'sql', '1c', 'irony', 'регулярный выражение', 'синтаксический анализатор', 'бухгалтерия', 'грамматика', 'кт', 'грамматический парсер', '.net', 'транслятор', 'грамматический разбор', '1с', 'sql', 'regexp', 'бухгалтерия программист', 'синтаксический анализ']
['аудиотехника', 'наушник', 'аудиомание', 'акустика', 'амплитудный частотный характеристика', 'миф реальность', 'аудиотехнология', 'аудиосистема', 'музыка']


Вроде видно паттерны. Основные -- Noun, Adj + Noun, PNoun, Noun + Noun. Однако это информация на глаз, а хорошо бы проверить с помощью такого же парсера, как и тот, которым будут размечаться тексты. Есть проблема -- русский и английский вместе. Было бы здорово, конечно, использовать какую-нибудь мультиязычную модель, но сначала давайте попробуем что-нибудь попроще. Если бы перед нами стоялазадача обработать текст, было бы, конечно, сложнее. Здесь можно попробовать всё, что содержит кириллицу, отдать парсеру для русского языка, а всё, что содержит латиницу -- парсеру для английского

In [74]:
def is_pattern(text, nlp, patterns):
    doc = nlp(text)
    stops = ['ADP', 'AUX', 'CCONJ', 'DET', 'INTJ', 'PART', 'PUNCT', 'SCONJ']
    pattern = []
    for sent in doc.sentences:
        for word in sent.words:
            if re.search('[\\\/&{}]+?', word.text):
                break
            else:
                if  word.upos not in stops:
                    pattern.append(word.upos)
                
    if pattern in patterns:
        return True
    else:
        return False

In [75]:
pattern_res = {}
patterns_en = [['PROPN'], ['PROPN', 'PROPN'], ['NOUN']]
patterns_ru = [['ADJ', 'NOUN'], ['NOUN', 'NOUN'], ['ADJ', 'ADJ', 'NOUN'], ['NOUN']]

In [79]:
def pattern_keywords(keywords, nlp_en=nlp_en, patterns_en=patterns_en, nlp_ru=nlp_ru, patterns_ru=patterns_ru):
    all_text_filter = []
    for text_keywords in keywords:
        text_keywords_filter = []
        for word in text_keywords:
            flag = False
            if re.search('[A-Za-z]+?', word[0]):
                flag = is_pattern(word[0], nlp_en, patterns_en)
            if re.search('[А-Яа-яЁё]+?', word[0]):
                flag = is_pattern(word[0], nlp_ru, patterns_ru)
            if flag is True:
                #print(word)
                text_keywords_filter.append(word)
        all_text_filter.append(text_keywords_filter)
    return all_text_filter

In [77]:
pattern_keywords(clean_texts_keywords['rake'])

('выбор консьюмер', 4.0)
('схожий имплементация', 4.0)
('environment', 1.0)
('smssent', 1.0)
('datetime', 1.0)
('start', 1.0)
('таба', 1.0)
('удобный # подключение', 4.5)
('grep repmgrd', 4.5)
('conf pg_ident', 4.0)
('conf postgresql', 4.0)
('нода', 1.5)
('ситуация', 1.0)
('auto', 1.0)
('repmgr_funcs', 1.0)
('pghost198', 1.0)
('метод', 1.0)
('ipv4', 1.0)
('node1', 1.0)
('пользователь', 1.0)
('табличка документ', 4.5)
('структура данные', 4.333333333333334)
('таблица бд', 4.0)
('бессонный цейтнот', 4.0)
('функция получитьстраненбазыдениябданиянный', 4.0)
('ряд преобразование', 4.0)
('пристальный взгляд', 4.0)
('правило грамматика', 4.0)
('большой вопрос', 4.0)
('метод узел', 4.0)
('клиент данные', 3.8333333333333335)
('клиент', 1.5)
('мысль', 1.0)
('жед', 1.0)
('организация', 1.0)
('контрагент', 1.0)
('язык', 1.0)
('сайт аудиомания', 4.5)
('полный катушка', 4.0)
('s', 1.0)


[[('выбор консьюмер', 4.0),
  ('схожий имплементация', 4.0),
  ('environment', 1.0),
  ('smssent', 1.0),
  ('datetime', 1.0),
  ('start', 1.0)],
 [('таба', 1.0)],
 [('удобный # подключение', 4.5),
  ('grep repmgrd', 4.5),
  ('conf pg_ident', 4.0),
  ('conf postgresql', 4.0),
  ('нода', 1.5),
  ('ситуация', 1.0),
  ('auto', 1.0),
  ('repmgr_funcs', 1.0),
  ('pghost198', 1.0),
  ('метод', 1.0),
  ('ipv4', 1.0),
  ('node1', 1.0),
  ('пользователь', 1.0)],
 [('табличка документ', 4.5),
  ('структура данные', 4.333333333333334),
  ('таблица бд', 4.0),
  ('бессонный цейтнот', 4.0),
  ('функция получитьстраненбазыдениябданиянный', 4.0),
  ('ряд преобразование', 4.0),
  ('пристальный взгляд', 4.0),
  ('правило грамматика', 4.0),
  ('большой вопрос', 4.0),
  ('метод узел', 4.0),
  ('клиент данные', 3.8333333333333335),
  ('клиент', 1.5),
  ('мысль', 1.0),
  ('жед', 1.0),
  ('организация', 1.0),
  ('контрагент', 1.0),
  ('язык', 1.0)],
 [('сайт аудиомания', 4.5), ('полный катушка', 4.0), ('s', 1

In [78]:
pattern_keywords(clean_texts_keywords['textrank'])

('сообщение', 0.37089808371824723)
('консьюмер', 0.21540568248167388)
('тип', 0.19213232105415434)
('команда событие', 0.1886906683440217)
('exchange', 0.17738269892650924)
('наименование', 0.12990296082773795)
('код', 0.11438941823881178)
('случай', 0.10718448571750971)
('конфигурация контейнер', 0.10258829363998175)
('masstransit', 0.10173403560110136)
('документация', 0.08985492376637803)
('ibus', 0.08975427852503873)
('обработка', 0.08937237401566817)
('данные', 0.08820661304809732)
('имя', 0.08001408757707883)
('bus', 0.0800095142583262)
('подключение', 0.07810810062540105)
('ibuscontrol', 0.077988055602667)
('работа', 0.07630926256849914)
('отправка', 0.0752540771918499)
('имплементация', 0.07257713357612683)
('действие', 0.07234591875724061)
('host', 0.07225306099830694)
('очередь', 0.06627265985827716)
('endpoint', 0.06625139504583179)
('rabbitmb', 0.0658506174328459)
('console', 0.06449407873370322)
('выполнение изображение', 0.06144243078900129)
('наследование', 0.06025449939

[[('сообщение', 0.37089808371824723),
  ('консьюмер', 0.21540568248167388),
  ('тип', 0.19213232105415434),
  ('команда событие', 0.1886906683440217),
  ('exchange', 0.17738269892650924),
  ('наименование', 0.12990296082773795),
  ('код', 0.11438941823881178),
  ('случай', 0.10718448571750971),
  ('конфигурация контейнер', 0.10258829363998175),
  ('masstransit', 0.10173403560110136),
  ('документация', 0.08985492376637803),
  ('ibus', 0.08975427852503873),
  ('обработка', 0.08937237401566817),
  ('данные', 0.08820661304809732),
  ('имя', 0.08001408757707883),
  ('bus', 0.0800095142583262),
  ('подключение', 0.07810810062540105),
  ('ibuscontrol', 0.077988055602667),
  ('работа', 0.07630926256849914),
  ('отправка', 0.0752540771918499),
  ('имплементация', 0.07257713357612683),
  ('действие', 0.07234591875724061),
  ('host', 0.07225306099830694),
  ('очередь', 0.06627265985827716),
  ('endpoint', 0.06625139504583179),
  ('rabbitmb', 0.0658506174328459),
  ('console', 0.06449407873370322

## Часть пятая. Точность, полнота, F-мера

In [76]:
def count_results(standard_keywords, auto_keywords):
    true_positive = 0
    false_positive = 0
    false_negative = 0
    for word in auto_keywords:
        if word in standard_keywords:
            true_positive += 1
        else:
            false_positive += 1
    for word in standard_keywords:
        if word not in auto_keywords:
            false_negative += 1
            
    return true_positive, false_positive, false_negative

In [68]:
def precision(tp, fp):
    return tp / (tp + fp)    


def recall(tp, fn):
    return tp / (tp + fn)


def f_score(tp, fp, fn):
    p = precision(tp, fp)
    r = recall(tp, fn)
    return 2*(p*r)/(p+r)

## Часть шестая. Ошибки и их анализ